# Ordinary Differential Equations

The solution of ODEs, particularly as initial value problems, is essential in numerical relativity. We start from finite differencing, where we know that (for example) the forward difference approximation to the first derivative is given by
$$
\left. f'_{\mathrm{FD}} \right|_{x_i} = \frac{f(x_{i+\Delta x}) - f(x_i)}{\Delta x} + \mathcal{O}(\Delta x).
$$

## Initial Value Problems

We focus on the IVP
$$
\frac{\mathrm{d}q}{\mathrm{d}t} = f(t, q), \quad q(t=0) = q_0,
$$
where $q$ is the *state vector* and $f$ is the *right-hand side* function. The solution is a function $q(t)$ that satisfies the ODE and the initial condition.

We can use two points of view to get Euler's method for this problem. The first, as seen, is to approximate the derivative at $t=T$ using forward differencing. The second is to integrate the ODE from $t=T$ to $t=T+\Delta t$, giving
$$
q(T + \Delta t) = q(T) + \int_T^{T+\Delta t} f(t, q) \, \mathrm{d}t.
$$
We can then approximate the integral in a number of ways. As we assume the solution is known at $t=T$ but not at later times, the most useful approximation is that the integral is approximately equal to $f(T, q(T)) \Delta t$. Both finite differencing and integration give the same result, which is Euler's method
$$
q(T + \Delta t) = q(T) + f(T, q(T)) \Delta t.
$$

### Implement and check

If we assume that pressure gradients are negligible then the equations of relativistic hydrodynamics can be written (with some choice of units) as
$$
\frac{\mathrm{d}}{\mathrm{d}t} \begin{pmatrix} \varepsilon \\ n \\ Y \end{pmatrix} = \begin{pmatrix} -(\varepsilon + P) \theta \\ -n \theta \\ -\tau^{-1} (Y - Y_{\mathrm{eq}}) \end{pmatrix}
$$
where $\varepsilon$ is the energy density, $n$ is the number density, $Y$ is a species fraction, $\theta$ is the expansion of the velocity, $P$ is the pressure, $\tau$ is the reaction timescale, and $Y_{\mathrm{eq}}$ is the equilibrium value of the species fraction.

Assuming that the pressure is given by $P = Y n \varepsilon$ and $Y_{\mathrm{eq}} = n / (3 \varepsilon)$, use Euler's method to solve up to $t=1$ for an expanding fluid where
$$
\begin{aligned}
n(0) &= 1, \\ \varepsilon(0) &= 1, \\ Y(0) &= 1, \\ \theta &= 10^{-1}, \\ \tau &= 10^{-1}.
\end{aligned}
$$

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import ipympl
import tqdm

In [ ]:
tau = 1e-1
theta = 1e-1

def P(eps, n, Y):
    return Y * n * eps

def Y_eq(eps, n, Y):
    return n / (3 * eps)

def rhs(t, q):
    eps, n, Y = q
    # Add your code for the RHS term here
    # replacing the "raise NotImplementedError" line.
    raise NotImplementedError("RHS not implemented yet")

def euler_step(f, t, q, dt):
    # Add your code for a single Euler step here
    # replacing the "raise NotImplementedError" line.
    raise NotImplementedError("Euler step not implemented yet")

# This evolves forward in time.
t, dt = np.linspace(0, 1, 10000, retstep=True)
q = np.zeros((len(t), 3))
q[0] = np.array([1, 1, 1])
for i in range(len(t)-1):
    q[i+1] = euler_step(rhs, t[i], q[i], dt)

In [ ]:
names = [r"$\varepsilon$", r"$n$", r"$Y$"]
fig, ax = plt.subplots(1, 3)
for i in range(3):
    ax[i].plot(t, q[:, i])
    ax[i].set_ylabel(names[i])
    ax[i].set_xlabel("t")
fig.tight_layout()
plt.show()

See what happens as the relaxation timescale is reduced. You should expect problems if the timescale is too short ($\tau \simeq \Delta t$).

In [ ]:
# Example small value of the timescale
tau = 3e-5
theta = 1e-1

# You should re-implement the RHS to pick up the new timescale
def rhs(t, q):
    eps, n, Y = q
    # Add your code for the RHS term here
    # replacing the "raise NotImplementedError" line.
    raise NotImplementedError("RHS not implemented yet")

t, dt = np.linspace(0, 1, 10000, retstep=True)
q = np.zeros((len(t), 3))
q[0] = np.array([1, 1, 1])
for i in range(len(t)-1):
    q[i+1] = euler_step(rhs, t[i], q[i], dt)

In [ ]:
# This will plot the results
# For small tau, expect it to blow up horribly.
names = [r"$\varepsilon$", r"$n$", r"$Y$"]
fig, ax = plt.subplots(1, 3)
for i in range(3):
    ax[i].plot(t, q[:, i])
    ax[i].set_ylabel(names[i])
    ax[i].set_xlabel("t")
fig.tight_layout()
plt.show()

## Stochastic DEs

Let us assume that the reactions driving the evolution of the species fraction $Y$ are stochastic. We then write the evolution equation as
$$
\mathrm{d} Y = -\tau^{-1} (Y - Y_{\mathrm{eq}}) \, \mathrm{d}t + \alpha \tau^{-1/2} (1 - Y_{\mathrm{eq}}) \, \mathrm{d}W,
$$
where $W$ is a Wiener process. The notation here is a weak form of the equation, so that $\mathrm{d}W$ is a stochastic increment that is zero on average and has variance $\mathrm{d}t$. The term $\alpha$ controls the strength of the noise.

The *Euler-Maruyama* method is the stochastic analogue of Euler's method. It is given by (in this case - the extension to more complex SDEs should follow the same pattern)
$$
Y(T + \Delta t) = Y(T) - \tau^{-1} (Y(T) - Y_{\mathrm{eq}}) \Delta t + \alpha \tau^{-1/2} (1 - Y_{\mathrm{eq}}) \Delta W,
$$
where $\Delta W$ is a random number drawn from a normal distribution with mean zero and variance $\Delta t$.

An individual evolution of an SDE is not so interesting. What is interesting is the ensemble behaviour. We can use the Euler-Maruyama method to evolve a large number of realisations of the SDE and then look at the ensemble average and variance.

### Implement and check

Using similar parameters to the first example (so $\tau = 10^{-1}$), with a noise parameter of $\alpha = 10^{-1}$, look at the behaviour of the stochastic case for a single realization, and for an ensemble of 1000 realizations. You should see that the ensemble average follows the deterministic solution, but that the variance is non-zero.

In [ ]:
tau = 1e-1
theta = 1e-1
alpha = 1e-1

def P(eps, n, Y):
    return Y * n * eps

def Y_eq(eps, n, Y):
    return n / (3 * eps)

def rhs(t, q):
    eps, n, Y = q
    return np.array([-(eps+P(eps, n, Y))*theta, -n*theta, -(Y - Y_eq(eps, n, Y))/tau])

def rhs_noise(t, q, dt):
    eps, n, Y = q
    # Add your code for the RHS term here
    # replacing the "raise NotImplementedError" line.
    # To get a normally distributed random variable for delta W
    # you can use np.random.normal() scaled by np.sqrt(tau*dt)
    raise NotImplementedError("RHS not implemented yet")

def euler_maruyama_step(f, t, q, dt):
    # Add your code for a single Euler step here
    # replacing the "raise NotImplementedError" line.
    # Remember that there are now two terms!
    raise NotImplementedError("Euler step not implemented yet")

t, dt = np.linspace(0, 1, 10000, retstep=True)
q = np.zeros((len(t), 3))
q[0] = np.array([1, 1, 1])
for i in range(len(t)-1):
    q[i+1] = euler_maruyama_step(rhs, t[i], q[i], dt)

In [ ]:
# Plot a single realization
names = [r"$\varepsilon$", r"$n$", r"$Y$"]
fig, ax = plt.subplots(1, 3)
for i in range(3):
    ax[i].plot(t, q[:, i])
    ax[i].set_ylabel(names[i])
    ax[i].set_xlabel("t")
fig.tight_layout()
plt.show()

In [ ]:
# Compute 1000 realizations. Due to the random walk,
# each will be different.
n_realizations = 1000
qs = np.zeros((n_realizations, len(t), 3))
qs[:, 0] = np.array([1, 1, 1])
for n in tqdm.tqdm(range(n_realizations)):
    for i in range(len(t)-1):
        qs[n, i+1] = euler_maruyama_step(rhs, t[i], qs[n, i], dt)

In [ ]:
# Plot the means
names = [r"$\varepsilon$", r"$n$", r"$Y$"]
fig, ax = plt.subplots(1, 3)
for i in range(3):
    ax[i].plot(t, np.mean(qs[:,:, i],axis=0))
    ax[i].set_ylabel(names[i])
    ax[i].set_xlabel("t")
fig.tight_layout()
plt.show()

In [ ]:
# Plot the variances
names = [r"$\varepsilon$", r"$n$", r"$Y$"]
fig, ax = plt.subplots(1, 3)
for i in range(3):
    ax[i].plot(t, np.var(qs[:,:, i],axis=0))
    ax[i].set_ylabel(names[i])
    ax[i].set_xlabel("t")
fig.tight_layout()
plt.show()

In [ ]:
# Plot essentially the range of all results
names = [r"$\varepsilon$", r"$n$", r"$Y$"]
fig, ax = plt.subplots(1, 3)
for i in range(3):
    ax[i].plot(t, np.mean(qs[:,:, i],axis=0))
    ax[i].fill_between(t, np.min(qs[:,:, i],axis=0), np.max(qs[:,:, i],axis=0), color='r', alpha=0.2)
    ax[i].set_ylabel(names[i])
    ax[i].set_xlabel("t")
fig.tight_layout()
plt.show()

## Boundary Value Problems

In a BVP, not all of the boundary conditions are imposed at the same point. This often occurs with constraints, or other non-local operations. Therefore these are not often used *for* evolution, but may be needed *during* evolution. Examples include the construction of the initial data, or finding the horizon of a black hole.

We will look at finding a black hole apparent horizon.

The spacetime we're going to look at is simplified:

* $3+1$ split (we're looking at one slice, so one instant in "time");
* axisymmetric (so we can consider only two dimensions in space, using $r, \theta$);
* "bitant" or "reflection" symmetric (so we only consider $\theta \in [0, \pi/2]$);
* all singularities have bare mass $1$;
* time-symmetric (the extrinsic curvature vanishes).

We then compute the expansion of outgoing null geodesics, and look for where this vanishes. The surface with radius $h(\theta)$ where this occurs is the apparent horizon. With our assumptions, $h$ obeys the boundary value problem

$$
  \frac{d^2 h}{d \theta^2} = 2 h + \frac{3}{h} \left( \frac{d h}{d \theta} \right)^2 + f \left( \theta, h, \frac{d h}{d \theta} \right), \qquad \frac{d h}{d \theta} ( \theta = 0 ) = 0 = \frac{d h}{d \theta} ( \theta = \pi/2 ).
$$

The function $f$ encodes the spacetime effects due to the singularities. 

To solve this problem we convert to *first order form*. Introduce the vector 
$$
  {\bf H} = \begin{pmatrix} h \\ \frac{d h}{d \theta} \end{pmatrix}.
$$
Then we have the problem
$$
  \frac{d}{d \theta} {\bf H} = {\bf F}({\bf H}, \theta) = \begin{pmatrix} H_2 \\ 2 H_1 + \frac{3}{H_1} H_2^2 + f(\theta, {\bf H}) \end{pmatrix}, \qquad H_2(\theta = 0) = 0 = H_2(\theta = \pi/2).
$$
We'll give the entire right-hand-side as code:

In [ ]:
def horizon_RHS(H, theta, z_singularities):
    """
    The RHS function for the apparent horizon problem.
    
    Parameters
    ----------
    
    H : array
        vector [h, dh/dtheta]
    theta : double
        angle
    z_singularities : array
        Location of the singularities on the z axis; non-negative
    
    Returns
    -------
    
    dHdtheta : array
        RHS
    """
    
    assert(np.all(np.array(z_singularities) >= 0.0)), "Location of singularities cannot be negative"
    
    h = H[0]
    dh = H[1]
    
    psi = 1.0
    dpsi_dr = 0.0
    dpsi_dtheta = 0.0
    for z in z_singularities:
        distance = np.sqrt((h*np.sin(theta))**2 + (h*np.cos(theta) - z)**2)
        psi += 0.5/distance
        dpsi_dr -= 0.5*(h-z*np.cos(theta))/distance**3
        dpsi_dtheta -= 0.5**h*z*np.sin(theta)/distance**3
        # Apply reflection symmetry
        if z > 0.0:
            distance = np.sqrt((h*np.sin(theta))**2 + (h*np.cos(theta) + z)**2)
            psi += 0.5/distance
            dpsi_dr -= 0.5*(h+z*np.cos(theta))/distance**3
            dpsi_dtheta += 0.5**h*z*np.sin(theta)/distance**3
            

    C2 = 1.0 / (1.0 + (dh / h)**2)
    # Impose that the term involving cot(theta) vanishes on axis.
    if (abs(theta) < 1e-16) or (abs(theta - np.pi) < 1e-16):
        cot_theta_dh_C2 = 0.0
    else:
        cot_theta_dh_C2 = dh / (np.tan(theta) * C2)
        
    dHdtheta = np.zeros_like(H)
    dHdtheta[0] = dh
    dHdtheta[1] = 2.0*h - cot_theta_dh_C2 + 4.0*h**2/(psi*C2)*(dpsi_dr - dpsi_dtheta*dh/h**2) + 3.0*dh**2/h
    
    return dHdtheta

### Shooting

The boundary conditions tell us the correct value of $H_2$ at $\theta = 0$. If we knew the correct value of $H_1$ at that point we could solve using a standard IVP solver, like the Euler method above. For example, in the case of a single Schwarzschild black hole the correct value is $H_1 = 1/2$. We can check this:

In [ ]:
# This relies on your euler_step code above!

def rhs_schwarzschild(theta, H):
    return horizon_RHS(H, theta, [0.0])

thetas, dtheta = np.linspace(0, np.pi/2, 1000, retstep=True)
Hs = np.zeros((len(thetas), 2))
Hs[0] = np.array([0.5, 0])
for i in range(len(thetas)-1):
    Hs[i+1] = euler_step(rhs_schwarzschild, thetas[i], Hs[i], dtheta)


In [ ]:
# Plot the results

fig = plt.figure()
ax = fig.add_subplot(221)
ax.plot(thetas, Hs[:, 0])
ax.set_ylabel(r"$h$")
ax.set_xlabel(r"$\theta$")
ax = fig.add_subplot(222)
ax.plot(thetas, Hs[:, 1])
ax.set_ylabel(r"$dh/d\theta$")
ax.set_xlabel(r"$\theta$")
ax = fig.add_subplot(223, polar=True)
ax.plot(thetas, Hs[:, 0])
ax = fig.add_subplot(224, polar=True)
ax.plot(thetas, Hs[:, 1])
fig.tight_layout()
plt.show()

This gives us a spherical horizon. However, if we had used a different value for $H_1(\theta=0)$ we would find that the required boundary condition on $H_2(\theta = \pi/2)$ would not be satisfied. Let us check this:

In [ ]:
Hs1 = np.zeros((len(thetas), 2))
Hs1[0] = np.array([0.55, 0])
Hs2 = np.zeros((len(thetas), 2))
Hs2[0] = np.array([0.45, 0])
for i in range(len(thetas)-1):
    Hs1[i+1] = euler_step(rhs_schwarzschild, thetas[i], Hs1[i], dtheta)
    Hs2[i+1] = euler_step(rhs_schwarzschild, thetas[i], Hs2[i], dtheta)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(221)
ax.plot(thetas, Hs1[:, 0], color='b')
ax.plot(thetas, Hs2[:, 0], color='r')
ax.set_ylabel(r"$h$")
ax.set_xlabel(r"$\theta$")
ax = fig.add_subplot(222)
ax.plot(thetas, Hs1[:, 1], color='b')
ax.plot(thetas, Hs2[:, 1], color='r')
ax.set_ylabel(r"$dh/d\theta$")
ax.set_xlabel(r"$\theta$")
ax = fig.add_subplot(223, polar=True)
ax.plot(thetas, Hs1[:, 0], color='b')
ax.plot(thetas, Hs2[:, 0], color='r')
ax = fig.add_subplot(224, polar=True)
ax.plot(thetas, Hs1[:, 1], color='b')
ax.plot(thetas, Hs2[:, 1], color='r')
fig.tight_layout()
plt.show()

This gives us the basic idea for the *shooting* method. As we don't know the correct value of $H_1$ at $\theta = 0$ we can guess a value, solve the IVP, and then check the boundary condition at $\theta = \pi/2$. We can then adjust our guess and repeat until we find the correct value. The question is how to adjust our guess.

#### Bisection

There are many different root-finding algorithms, but the simplest is bisection. Here we asssume that we know some guess that is "too small", and some guess that is "too large". We then take the midpoint of these two guesses and check the boundary condition. If the boundary condition is satisfied then we replace the "too small" guess with the midpoint, otherwise we replace the "too large" guess. We then repeat until the two guesses are close enough.

To make this more precise, in our case we define a function
$$
G[H_1(0)] = H_2(\pi/2; H_1(0)) - 0.
$$
This compares the value of $H_2$ at $\theta = \pi/2$, for a given *initial* value of $H_1$ at $\theta = 0$, with the required value of $0$. We saw above that where the initial value of $H_1$ is too large then $G > 0$, and where the initial value of $H_1$ is too small then $G < 0$. We therefore know that the correct value is somewhere in between. We guess that a better value is the midpoint of the two guesses. If the midpoint gives a value of $G$ that is positive then it forms a new upper bound; otherwise, it forms a lower bound. We then repeat until the bounds are close enough.

Here we check this with Schwarzschild, using initial bounds of $0.4$, $0.55$ (so the midpoint does not land precisely on the right answer).

In [ ]:
H1_lo = 0.4
H1_hi = 0.55
tol = 1e-3
# Implement bisection here.
# Don't use a function (yet), just a simple loop
# Replace the NotImplemented line as usual
raise NotImplementedError("Bisection not implemented yet")
# Then print the result
print(f"Bisection gives H_1(0) = {(H1_lo + H1_hi)/2} ([{H1_lo}, {H1_hi}])")

### Implement and check

We can now solve the binary black hole case. Try putting the singularities at $z = \pm 0.75$ using `z_singularity = [0.75]`. A good guess for the initial value of the radius is $H_1(0) \in [1.26, 1.3]$.

The final solution can be compared to [this paper](http://arxiv.org/abs/gr-qc/9809004). Symmetry means that the solution for $h$, $H_1$, constructed for $\theta \in [0, \pi/2]$, can be plotted against the three intervals $[\pi, \pi/2], [\pi, 3 \pi/2]$, and $[2 \pi, 3 \pi/2]$ (note the ordering!) to get the full surface.

In [ ]:
# The RHS directly uses the given function
def rhs_binary(theta, H):
    return horizon_RHS(H, theta, [0.75])

H1_lo = 1.26
H1_hi = 1.3
tol = 1e-4
# Implement bisection here.
# Don't use a function (yet), just a simple loop
# Replace the NotImplemented line as usual
raise NotImplementedError("Bisection not implemented yet")
# Then print the result
print(f"Bisection gives H_1(0) = {(H1_lo + H1_hi)/2} ([{H1_lo}, {H1_hi}])")

In [ ]:
# Now the result above can be used, alongside euler_step again,
# to get the "true" horizon
thetas, dtheta = np.linspace(0, np.pi/2, 1000, retstep=True)
Hs = np.zeros((len(thetas), 2))
Hs[0] = np.array([(H1_lo + H1_hi)/2, 0])
for i in range(len(thetas)-1):
    Hs[i+1] = euler_step(rhs_binary, thetas[i], Hs[i], dtheta)

# Plot the result
fig = plt.figure()
ax = fig.add_subplot(221)
ax.plot(thetas, Hs[:, 0])
ax.set_ylabel(r"$h$")
ax.set_xlabel(r"$\theta$")
ax = fig.add_subplot(222)
ax.plot(thetas, Hs[:, 1])
ax.set_ylabel(r"$dh/d\theta$")
ax.set_xlabel(r"$\theta$")
ax = fig.add_subplot(223, polar=True)
ax.plot(thetas, Hs[:, 0])
ax = fig.add_subplot(224, polar=True)
ax.plot(thetas, Hs[:, 1])
fig.tight_layout()
plt.show()

In [ ]:
# This produces a plot of the "full" horizon using the symmetries.
plt.figure()
plt.polar(thetas, Hs[:, 0], color='k')
plt.polar(2*np.pi-thetas, Hs[:, 0], color='k')
plt.polar(np.pi-thetas, Hs[:, 0], color='k')
plt.polar(np.pi+thetas, Hs[:, 0], color='k')
plt.show()